# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv("weather_data_1.csv")
weather_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,wind Speed,Country,Date
0,0,Esperance,-33.8667,121.9000,51.64,58,0,13.42,AU,1632184808
1,1,San Patricio,28.0170,-97.5169,90.01,67,0,12.10,US,1632184999
2,2,Barrow,71.2906,-156.7887,33.82,64,90,14.97,US,1632184999
3,3,Nome,64.5011,-165.4064,37.47,65,90,16.11,US,1632185000
4,4,Miandrivazo,-19.5167,45.4667,73.29,75,95,3.96,MG,1632185000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [145]:
gmaps.configure(api_key = g_key)

locations = weather_data_df[['Lat', 'Lng']]
rating = weather_data_df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = rating,
                                dissipating = False, max_intensity = 30,
                                point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [118]:
# temp between 70 and 80F
narrowed_df = weather_data_df[(weather_data_df['Max Temp'] >= 70) & 
                              (weather_data_df['Max Temp'] <=80) & 
                              (weather_data_df['wind Speed'] <= 10) & 
                              (weather_data_df['Cloudiness'] != 0)]
narrowed_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,wind Speed,Country,Date
4,4,Miandrivazo,-19.5167,45.4667,73.29,75,95,3.96,MG,1632185000
20,20,Maragogi,-9.0122,-35.2225,73.99,75,18,6.44,BR,1632184942
31,31,Pratāpgarh,24.0333,74.7833,72.45,97,95,9.82,IN,1632185010
34,34,Morristown,36.2140,-83.2949,72.70,78,90,6.91,US,1632184722
38,38,Bengkulu,-3.8004,102.2655,78.15,83,86,3.18,ID,1632184821
...,...,...,...,...,...,...,...,...,...,...
516,516,Galle,6.0367,80.2170,79.25,84,87,8.37,LK,1632185037
530,530,Bambanglipuro,-7.9500,110.2833,79.63,76,100,4.72,ID,1632185211
541,541,Caravelas,-17.7125,-39.2481,72.05,84,5,4.41,BR,1632185216
542,542,Meulaboh,4.1363,96.1285,77.41,83,99,1.83,ID,1632185216


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = narrowed_df
# hotel_df['Hotel Name']="" there is a bug that prevents me from entering leaving this line
params = {
    "radius": 5000,
    "type": 'hotel',
    'key': g_key
}

for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    
    params['location'] = f"{latitude},{longitude}"
    hotels_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    hotels = requests.get(hotels_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        Pass
    



In [139]:
hotel_df.head(25)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,wind Speed,Country,Date,Hotel Name
4,4,Miandrivazo,-19.5167,45.4667,73.29,75,95,3.96,MG,1632185000,Miandrivazo
20,20,Maragogi,-9.0122,-35.2225,73.99,75,18,6.44,BR,1632184942,Maragogi
31,31,Pratāpgarh,24.0333,74.7833,72.45,97,95,9.82,IN,1632185010,Pratapgarh
34,34,Morristown,36.2140,-83.2949,72.70,78,90,6.91,US,1632184722,Morristown
38,38,Bengkulu,-3.8004,102.2655,78.15,83,86,3.18,ID,1632184821,Bengkulu
42,42,Hamilton,39.1834,-84.5333,72.79,88,90,5.75,US,1632185014,Cincinnati
46,46,Sinnamary,5.3833,-52.9500,79.23,85,22,8.01,GF,1632184881,Sinnamary
54,54,George Town,5.4112,100.3354,78.73,94,20,4.61,MY,1632184765,George Town
74,74,Wawa,9.9022,4.4192,75.63,87,100,5.08,NG,1632185026,Wawa
78,78,Eureka,40.8021,-124.1637,72.21,69,1,4.38,US,1632185028,Eureka


In [140]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [147]:
# Add marker layer ontop of heat map
label_layer = gmaps.symbol_layer(locations, hotel_info)
fig.add_layer(label_layer)

fig

# Display figure


AssertionError: 